# 🚀 Prédiction de Churn Améliorée + Soumission Kaggle

Ce notebook utilise les **insights de la segmentation des churners** pour améliorer les performances de prédiction.

## Stratégies d'amélioration implémentées

1. **Features de proximité aux profils** - Distance aux centroïdes des churners
2. **Scores composites** - Frustration, désengagement, risque
3. **Features temporelles avancées** - Tendances sur plusieurs périodes
4. **Comparaison avant/après** - Mesure de l'amélioration

---

## 1. Setup et Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, classification_report, roc_curve, precision_recall_curve
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

# Constantes
REFERENCE_DATE = pd.to_datetime('2018-11-20')
RANDOM_STATE = 42

print("✅ Imports chargés")

## 2. Chargement des Données

In [ ]:
# Charger les données d'entraînement
df_train = pd.read_csv('df_train_sample.csv')
df_train['time'] = pd.to_datetime(df_train['time'])
df_train['registration'] = pd.to_datetime(df_train['registration'])

print(f"📊 Données d'entraînement: {len(df_train):,} événements")
print(f"👥 Utilisateurs uniques: {df_train['userId'].nunique():,}")

## 3. Feature Engineering - Version de Base

In [ ]:
def create_base_features(df, reference_date):
    """
    Crée les features de base (version initiale sans améliorations).
    """
    df_filtered = df[df['time'] <= reference_date].copy()
    
    user_features = pd.DataFrame()
    user_features['userId'] = df_filtered['userId'].unique()
    
    # Info utilisateur
    user_info = df_filtered.groupby('userId').agg({
        'gender': 'first',
        'level': 'last',
        'registration': 'first',
        'will_churn_10days': 'first'
    }).reset_index()
    user_features = user_features.merge(user_info, on='userId', how='left')
    user_features['days_since_registration'] = (reference_date - user_features['registration']).dt.days
    
    # Activité globale
    activity = df_filtered.groupby('userId').agg({
        'ts': 'count',
        'sessionId': 'nunique',
        'time': ['min', 'max'],
        'length': ['sum', 'mean', 'count'],
    })
    activity.columns = ['total_events', 'unique_sessions', 'first_activity', 'last_activity',
                        'total_listening_time', 'avg_song_length', 'songs_with_length']
    activity = activity.reset_index()
    user_features = user_features.merge(activity, on='userId', how='left')
    
    # Métriques temporelles
    user_features['days_since_last_activity'] = (reference_date - pd.to_datetime(user_features['last_activity'])).dt.days
    user_features['activity_span_days'] = (pd.to_datetime(user_features['last_activity']) - pd.to_datetime(user_features['first_activity'])).dt.days + 1
    user_features['events_per_day'] = (user_features['total_events'] / user_features['activity_span_days']).replace([np.inf, -np.inf], 0)
    user_features['sessions_per_day'] = (user_features['unique_sessions'] / user_features['activity_span_days']).replace([np.inf, -np.inf], 0)
    
    # Pages visitées
    important_pages = ['NextSong', 'Home', 'Thumbs Up', 'Thumbs Down', 'Add to Playlist', 
                       'Add Friend', 'Roll Advert', 'Downgrade', 'Cancel', 'Submit Downgrade', 
                       'Error', 'Help', 'Settings', 'Logout', 'Upgrade']
    
    page_counts = df_filtered.groupby(['userId', 'page']).size().unstack(fill_value=0)
    for page in important_pages:
        col_name = f'page_{page.lower().replace(" ", "_")}'
        if page in page_counts.columns:
            temp = page_counts[[page]].reset_index()
            temp.columns = ['userId', col_name]
            user_features = user_features.merge(temp, on='userId', how='left')
            user_features[col_name] = user_features[col_name].fillna(0)
        else:
            user_features[col_name] = 0
    
    # Features dérivées de base
    user_features['churn_signals'] = (user_features.get('page_downgrade', 0) + 
                                       user_features.get('page_cancel', 0) + 
                                       user_features.get('page_submit_downgrade', 0))
    user_features['thumbs_ratio'] = user_features.get('page_thumbs_down', 0) / (user_features.get('page_thumbs_up', 0) + 1)
    user_features['error_rate'] = (user_features.get('page_error', 0) / user_features['total_events']).fillna(0)
    user_features['song_event_ratio'] = (user_features.get('page_nextsong', 0) / user_features['total_events']).fillna(0)
    user_features['positive_interactions'] = (user_features.get('page_thumbs_up', 0) + 
                                               user_features.get('page_add_to_playlist', 0) + 
                                               user_features.get('page_add_friend', 0))
    user_features['positive_interaction_rate'] = (user_features['positive_interactions'] / user_features['total_events']).fillna(0)
    
    # Dernière semaine
    one_week_before = reference_date - pd.Timedelta(days=7)
    last_week = df_filtered[df_filtered['time'] >= one_week_before].groupby('userId').size().reset_index()
    last_week.columns = ['userId', 'events_last_week']
    user_features = user_features.merge(last_week, on='userId', how='left')
    user_features['events_last_week'] = user_features['events_last_week'].fillna(0)
    user_features['activity_trend'] = user_features['events_last_week'] / (user_features['total_events'] + 1)
    
    return user_features

In [ ]:
# Créer les features de base
user_df_base = create_base_features(df_train, REFERENCE_DATE)
print(f"✅ Features de base créées: {user_df_base.shape}")

## 4. Feature Engineering - Version Améliorée 🚀

### Améliorations basées sur l'analyse de segmentation :
1. **Scores composites** (frustration, désengagement)
2. **Features temporelles multi-périodes** (3j, 7j, 14j)
3. **Distance aux profils de churners**
4. **Indicateurs de risque**

In [ ]:
def create_enhanced_features(df, reference_date):
    """
    Crée les features améliorées basées sur l'analyse de segmentation.
    """
    df_filtered = df[df['time'] <= reference_date].copy()
    
    # Commencer avec les features de base
    user_features = create_base_features(df, reference_date)
    
    # =========================================================================
    # AMÉLIORATION 1: SCORES COMPOSITES
    # =========================================================================
    
    # Score de FRUSTRATION (insight: les churners frustrés ont thumbs_ratio élevé + aide + erreurs)
    user_features['frustration_score'] = (
        user_features['thumbs_ratio'] * 2 +  # Poids double pour le mécontentement
        (user_features.get('page_help', 0) / (user_features['total_events'] + 1)) * 3 +
        user_features['error_rate'] * 2 +
        (user_features.get('page_downgrade', 0) > 0).astype(int) * 5  # Signal fort
    )
    
    # Score de DÉSENGAGEMENT (insight: 52% des churners sont des "désengagés progressifs")
    # Normaliser days_since_last_activity
    max_inactivity = user_features['days_since_last_activity'].quantile(0.95)
    user_features['disengagement_score'] = (
        (user_features['days_since_last_activity'] / (max_inactivity + 1)).clip(0, 1) * 3 +
        (1 - user_features['activity_trend'].clip(0, 1)) * 2 +
        (1 - user_features['positive_interaction_rate'].clip(0, 1)) * 1
    )
    
    # Score de RISQUE GLOBAL (combinaison)
    user_features['risk_score'] = (
        user_features['frustration_score'] * 0.4 +
        user_features['disengagement_score'] * 0.6  # Plus de poids car 52% des churners
    )
    
    # =========================================================================
    # AMÉLIORATION 2: FEATURES TEMPORELLES MULTI-PÉRIODES
    # =========================================================================
    
    periods = [3, 7, 14, 21]  # Jours
    
    for days in periods:
        period_start = reference_date - pd.Timedelta(days=days)
        period_data = df_filtered[df_filtered['time'] >= period_start]
        
        # Événements dans la période
        period_events = period_data.groupby('userId').size().reset_index()
        period_events.columns = ['userId', f'events_last_{days}d']
        user_features = user_features.merge(period_events, on='userId', how='left')
        user_features[f'events_last_{days}d'] = user_features[f'events_last_{days}d'].fillna(0)
        
        # Sessions dans la période
        period_sessions = period_data.groupby('userId')['sessionId'].nunique().reset_index()
        period_sessions.columns = ['userId', f'sessions_last_{days}d']
        user_features = user_features.merge(period_sessions, on='userId', how='left')
        user_features[f'sessions_last_{days}d'] = user_features[f'sessions_last_{days}d'].fillna(0)
    
    # Tendances (comparaison entre périodes)
    user_features['trend_7d_vs_14d'] = (
        user_features['events_last_7d'] / (user_features['events_last_14d'] - user_features['events_last_7d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    user_features['trend_3d_vs_7d'] = (
        user_features['events_last_3d'] / (user_features['events_last_7d'] - user_features['events_last_3d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    # Accélération du désengagement
    user_features['activity_acceleration'] = user_features['trend_3d_vs_7d'] - user_features['trend_7d_vs_14d']
    
    # =========================================================================
    # AMÉLIORATION 3: INDICATEURS BINAIRES DE RISQUE
    # =========================================================================
    
    # Basé sur les insights de la segmentation
    user_features['is_inactive_7d'] = (user_features['events_last_7d'] == 0).astype(int)
    user_features['is_inactive_14d'] = (user_features['events_last_14d'] == 0).astype(int)
    user_features['is_inactive_21d'] = (user_features['events_last_21d'] == 0).astype(int)
    
    user_features['has_visited_downgrade'] = (user_features.get('page_downgrade', 0) > 0).astype(int)
    user_features['has_visited_cancel'] = (user_features.get('page_cancel', 0) > 0).astype(int)
    user_features['has_visited_help'] = (user_features.get('page_help', 0) > 0).astype(int)
    user_features['has_errors'] = (user_features.get('page_error', 0) > 0).astype(int)
    
    user_features['is_low_engagement'] = (user_features['positive_interactions'] < 1).astype(int)
    user_features['is_declining'] = (user_features['trend_7d_vs_14d'] < 0.5).astype(int)
    
    # =========================================================================
    # AMÉLIORATION 4: FEATURES D'INTERACTION
    # =========================================================================
    
    # Interaction niveau x activité
    user_features['level_encoded'] = LabelEncoder().fit_transform(user_features['level'].fillna('unknown'))
    user_features['paid_x_inactive'] = user_features['level_encoded'] * user_features['days_since_last_activity']
    user_features['paid_x_lowengagement'] = user_features['level_encoded'] * user_features['is_low_engagement']
    
    # Interaction ancienneté x activité
    user_features['tenure_x_inactive'] = user_features['days_since_registration'] * user_features['is_inactive_14d']
    
    return user_features

In [ ]:
# Créer les features améliorées
user_df_enhanced = create_enhanced_features(df_train, REFERENCE_DATE)
print(f"✅ Features améliorées créées: {user_df_enhanced.shape}")
print(f"\n📊 Nouvelles features ajoutées: {user_df_enhanced.shape[1] - user_df_base.shape[1]}")

## 5. Ajout des Features de Distance aux Profils de Churners

In [ ]:
def add_profile_distance_features(user_df, n_clusters=4):
    """
    Ajoute les features de distance aux centroïdes des profils de churners.
    """
    # Features pour le clustering
    clustering_features = [
        'total_events', 'unique_sessions', 'days_since_last_activity',
        'events_per_day', 'positive_interactions', 'thumbs_ratio',
        'page_help', 'error_rate', 'days_since_registration', 'activity_trend'
    ]
    clustering_features = [f for f in clustering_features if f in user_df.columns]
    
    # Filtrer les churners pour créer les profils
    churners = user_df[user_df['will_churn_10days'] == 1].copy()
    
    if len(churners) < n_clusters:
        print("⚠️ Pas assez de churners pour créer les profils")
        return user_df
    
    # Normaliser
    scaler = StandardScaler()
    X_churners = churners[clustering_features].fillna(0)
    X_churners_scaled = scaler.fit_transform(X_churners)
    
    # Créer les clusters de churners
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_churners_scaled)
    
    # Calculer les distances pour TOUS les utilisateurs
    X_all = user_df[clustering_features].fillna(0)
    X_all_scaled = scaler.transform(X_all)
    
    # Distance à chaque centroïde
    distances = cdist(X_all_scaled, kmeans.cluster_centers_, metric='euclidean')
    
    for i in range(n_clusters):
        user_df[f'dist_to_churner_profile_{i}'] = distances[:, i]
    
    # Distance minimale (au profil le plus proche)
    user_df['min_dist_to_churner'] = distances.min(axis=1)
    
    # Profil de churner le plus proche
    user_df['closest_churner_profile'] = distances.argmin(axis=1)
    
    # Probabilité "soft" d'appartenir à chaque profil (inverse de la distance normalisée)
    dist_sum = distances.sum(axis=1, keepdims=True)
    proba_profiles = 1 - (distances / (dist_sum + 0.001))
    proba_profiles = proba_profiles / proba_profiles.sum(axis=1, keepdims=True)
    
    for i in range(n_clusters):
        user_df[f'proba_churner_profile_{i}'] = proba_profiles[:, i]
    
    # Score de similarité maximal avec un profil de churner
    user_df['max_churner_similarity'] = proba_profiles.max(axis=1)
    
    print(f"✅ Features de distance aux {n_clusters} profils de churners ajoutées")
    
    return user_df

In [ ]:
# Ajouter les features de distance
user_df_enhanced = add_profile_distance_features(user_df_enhanced, n_clusters=4)
print(f"\n📊 Total features: {user_df_enhanced.shape[1]}")

## 6. Comparaison des Performances: Base vs Amélioré

In [ ]:
def prepare_features(user_df, exclude_cols=None):
    """
    Prépare les features pour la modélisation.
    """
    if exclude_cols is None:
        exclude_cols = ['userId', 'will_churn_10days', 'registration', 
                        'first_activity', 'last_activity', 'gender', 'level']
    
    # Sélectionner les colonnes numériques
    feature_cols = [col for col in user_df.columns 
                    if col not in exclude_cols 
                    and user_df[col].dtype in ['int64', 'float64']]
    
    X = user_df[feature_cols].fillna(0)
    y = user_df['will_churn_10days']
    
    return X, y, feature_cols


def evaluate_model(X, y, model_name="Model"):
    """
    Évalue un modèle avec cross-validation.
    """
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )
    
    # Normaliser
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modèles
    models = {
        'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=RANDOM_STATE),
        'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=RANDOM_STATE)
    }
    
    results = {}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    for name, model in models.items():
        if name == 'Logistic Regression':
            X_tr, X_te = X_train_scaled, X_test_scaled
        else:
            X_tr, X_te = X_train, X_test
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_tr, y_train, cv=cv, scoring='roc_auc')
        
        # Fit et prédiction
        model.fit(X_tr, y_train)
        y_pred_proba = model.predict_proba(X_te)[:, 1]
        y_pred = model.predict(X_te)
        
        results[name] = {
            'cv_auc': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'test_auc': roc_auc_score(y_test, y_pred_proba),
            'test_f1': f1_score(y_test, y_pred),
            'model': model,
            'scaler': scaler
        }
    
    return results, X_test, y_test

In [ ]:
# Évaluation avec features de BASE
print("="*60)
print("📊 ÉVALUATION - FEATURES DE BASE")
print("="*60)

X_base, y_base, features_base = prepare_features(user_df_base)
print(f"Nombre de features: {len(features_base)}")

results_base, _, _ = evaluate_model(X_base, y_base, "Base")

print("\nRésultats:")
for name, res in results_base.items():
    print(f"  {name}: CV AUC={res['cv_auc']:.4f} | Test AUC={res['test_auc']:.4f} | F1={res['test_f1']:.4f}")

In [ ]:
# Évaluation avec features AMÉLIORÉES
print("\n" + "="*60)
print("🚀 ÉVALUATION - FEATURES AMÉLIORÉES")
print("="*60)

X_enhanced, y_enhanced, features_enhanced = prepare_features(user_df_enhanced)
print(f"Nombre de features: {len(features_enhanced)}")

results_enhanced, X_test_final, y_test_final = evaluate_model(X_enhanced, y_enhanced, "Enhanced")

print("\nRésultats:")
for name, res in results_enhanced.items():
    print(f"  {name}: CV AUC={res['cv_auc']:.4f} | Test AUC={res['test_auc']:.4f} | F1={res['test_f1']:.4f}")

In [ ]:
# Comparaison des améliorations
print("\n" + "="*60)
print("📈 AMÉLIORATION DES PERFORMANCES")
print("="*60)

comparison_data = []
for model_name in results_base.keys():
    base_auc = results_base[model_name]['test_auc']
    enhanced_auc = results_enhanced[model_name]['test_auc']
    improvement = (enhanced_auc - base_auc) / base_auc * 100
    
    comparison_data.append({
        'Modèle': model_name,
        'AUC Base': base_auc,
        'AUC Amélioré': enhanced_auc,
        'Amélioration (%)': improvement
    })
    
    print(f"{model_name}:")
    print(f"  Base: {base_auc:.4f} → Amélioré: {enhanced_auc:.4f} ({improvement:+.2f}%)")

comparison_df = pd.DataFrame(comparison_data)
comparison_df

In [ ]:
# Visualisation de la comparaison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Barplot comparatif
ax1 = axes[0]
x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax1.bar(x - width/2, comparison_df['AUC Base'], width, label='Base', color='#3498db')
bars2 = ax1.bar(x + width/2, comparison_df['AUC Amélioré'], width, label='Amélioré', color='#2ecc71')

ax1.set_ylabel('AUC Score')
ax1.set_title('Comparaison AUC: Base vs Amélioré', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(['LogReg', 'RF', 'GB'])
ax1.legend()
ax1.set_ylim(0.7, 0.85)
ax1.grid(True, alpha=0.3)

# Ajouter les valeurs sur les barres
for bar in bars1:
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)

# 2. Amélioration en %
ax2 = axes[1]
colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in comparison_df['Amélioration (%)']]
bars = ax2.bar(comparison_df['Modèle'], comparison_df['Amélioration (%)'], color=colors)
ax2.set_ylabel('Amélioration (%)')
ax2.set_title('Gain de Performance (%)', fontsize=13, fontweight='bold')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3)

for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.1, 
             f'{height:+.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Analyse des Features les Plus Importantes

In [ ]:
# Feature importance du meilleur modèle (Gradient Boosting ou Random Forest)
best_model_name = max(results_enhanced.keys(), key=lambda k: results_enhanced[k]['test_auc'])
best_model = results_enhanced[best_model_name]['model']

print(f"🏆 Meilleur modèle: {best_model_name} (AUC={results_enhanced[best_model_name]['test_auc']:.4f})")

# Feature importance
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': features_enhanced,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n📊 Top 20 Features les Plus Importantes:")
    print(importance_df.head(20).to_string(index=False))

In [ ]:
# Visualisation des features importantes
fig, ax = plt.subplots(figsize=(10, 10))

top_20 = importance_df.head(20)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(top_20)))

ax.barh(range(len(top_20)), top_20['importance'], color=colors)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['feature'])
ax.invert_yaxis()
ax.set_xlabel('Importance')
ax.set_title(f'Top 20 Features ({best_model_name})', fontsize=13, fontweight='bold')
ax.grid(True, axis='x', alpha=0.3)

# Highlight les nouvelles features
new_features = ['frustration_score', 'disengagement_score', 'risk_score', 
                'min_dist_to_churner', 'max_churner_similarity',
                'trend_7d_vs_14d', 'trend_3d_vs_7d', 'activity_acceleration']

for i, feat in enumerate(top_20['feature']):
    if any(nf in feat for nf in new_features) or 'dist_to_churner' in feat or 'proba_churner' in feat:
        ax.get_yticklabels()[i].set_color('green')
        ax.get_yticklabels()[i].set_fontweight('bold')

plt.tight_layout()
plt.savefig('feature_importance_enhanced.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💚 Les features en vert sont les NOUVELLES features ajoutées grâce à l'analyse de segmentation")

## 8. Entraînement du Modèle Final

In [ ]:
# Entraîner le modèle final sur TOUTES les données
print("🎯 Entraînement du modèle final sur l'ensemble des données...")

# Utiliser le meilleur modèle
final_model = GradientBoostingClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE
)

# Scaler final
final_scaler = StandardScaler()
X_all = user_df_enhanced[features_enhanced].fillna(0)
y_all = user_df_enhanced['will_churn_10days']

# Fit sur toutes les données
final_model.fit(X_all, y_all)

print(f"✅ Modèle final entraîné sur {len(X_all)} utilisateurs")
print(f"📊 Features utilisées: {len(features_enhanced)}")

---

# 📤 9. SOUMISSION KAGGLE

Cette section permet de générer le fichier de soumission à partir du dataset de test.

In [ ]:
def create_features_for_submission(df, reference_date):
    """
    Crée les features pour le dataset de test (sans la colonne will_churn_10days).
    Version adaptée pour la soumission.
    """
    df_filtered = df[df['time'] <= reference_date].copy()
    
    user_features = pd.DataFrame()
    user_features['userId'] = df_filtered['userId'].unique()
    
    # Info utilisateur (sans will_churn_10days)
    user_info = df_filtered.groupby('userId').agg({
        'gender': 'first',
        'level': 'last',
        'registration': 'first',
    }).reset_index()
    user_features = user_features.merge(user_info, on='userId', how='left')
    user_features['days_since_registration'] = (reference_date - user_features['registration']).dt.days
    
    # Activité globale
    activity = df_filtered.groupby('userId').agg({
        'ts': 'count',
        'sessionId': 'nunique',
        'time': ['min', 'max'],
        'length': ['sum', 'mean', 'count'],
    })
    activity.columns = ['total_events', 'unique_sessions', 'first_activity', 'last_activity',
                        'total_listening_time', 'avg_song_length', 'songs_with_length']
    activity = activity.reset_index()
    user_features = user_features.merge(activity, on='userId', how='left')
    
    # Métriques temporelles
    user_features['days_since_last_activity'] = (reference_date - pd.to_datetime(user_features['last_activity'])).dt.days
    user_features['activity_span_days'] = (pd.to_datetime(user_features['last_activity']) - pd.to_datetime(user_features['first_activity'])).dt.days + 1
    user_features['events_per_day'] = (user_features['total_events'] / user_features['activity_span_days']).replace([np.inf, -np.inf], 0)
    user_features['sessions_per_day'] = (user_features['unique_sessions'] / user_features['activity_span_days']).replace([np.inf, -np.inf], 0)
    
    # Pages visitées
    important_pages = ['NextSong', 'Home', 'Thumbs Up', 'Thumbs Down', 'Add to Playlist', 
                       'Add Friend', 'Roll Advert', 'Downgrade', 'Cancel', 'Submit Downgrade', 
                       'Error', 'Help', 'Settings', 'Logout', 'Upgrade']
    
    page_counts = df_filtered.groupby(['userId', 'page']).size().unstack(fill_value=0)
    for page in important_pages:
        col_name = f'page_{page.lower().replace(" ", "_")}'
        if page in page_counts.columns:
            temp = page_counts[[page]].reset_index()
            temp.columns = ['userId', col_name]
            user_features = user_features.merge(temp, on='userId', how='left')
            user_features[col_name] = user_features[col_name].fillna(0)
        else:
            user_features[col_name] = 0
    
    # Features dérivées
    user_features['churn_signals'] = (user_features.get('page_downgrade', 0) + 
                                       user_features.get('page_cancel', 0) + 
                                       user_features.get('page_submit_downgrade', 0))
    user_features['thumbs_ratio'] = user_features.get('page_thumbs_down', 0) / (user_features.get('page_thumbs_up', 0) + 1)
    user_features['error_rate'] = (user_features.get('page_error', 0) / user_features['total_events']).fillna(0)
    user_features['song_event_ratio'] = (user_features.get('page_nextsong', 0) / user_features['total_events']).fillna(0)
    user_features['positive_interactions'] = (user_features.get('page_thumbs_up', 0) + 
                                               user_features.get('page_add_to_playlist', 0) + 
                                               user_features.get('page_add_friend', 0))
    user_features['positive_interaction_rate'] = (user_features['positive_interactions'] / user_features['total_events']).fillna(0)
    
    # Dernière semaine
    one_week_before = reference_date - pd.Timedelta(days=7)
    last_week = df_filtered[df_filtered['time'] >= one_week_before].groupby('userId').size().reset_index()
    last_week.columns = ['userId', 'events_last_week']
    user_features = user_features.merge(last_week, on='userId', how='left')
    user_features['events_last_week'] = user_features['events_last_week'].fillna(0)
    user_features['activity_trend'] = user_features['events_last_week'] / (user_features['total_events'] + 1)
    
    # =========================================================================
    # FEATURES AMÉLIORÉES
    # =========================================================================
    
    # Scores composites
    user_features['frustration_score'] = (
        user_features['thumbs_ratio'] * 2 +
        (user_features.get('page_help', 0) / (user_features['total_events'] + 1)) * 3 +
        user_features['error_rate'] * 2 +
        (user_features.get('page_downgrade', 0) > 0).astype(int) * 5
    )
    
    max_inactivity = user_features['days_since_last_activity'].quantile(0.95)
    user_features['disengagement_score'] = (
        (user_features['days_since_last_activity'] / (max_inactivity + 1)).clip(0, 1) * 3 +
        (1 - user_features['activity_trend'].clip(0, 1)) * 2 +
        (1 - user_features['positive_interaction_rate'].clip(0, 1)) * 1
    )
    
    user_features['risk_score'] = (
        user_features['frustration_score'] * 0.4 +
        user_features['disengagement_score'] * 0.6
    )
    
    # Features temporelles multi-périodes
    periods = [3, 7, 14, 21]
    for days in periods:
        period_start = reference_date - pd.Timedelta(days=days)
        period_data = df_filtered[df_filtered['time'] >= period_start]
        
        period_events = period_data.groupby('userId').size().reset_index()
        period_events.columns = ['userId', f'events_last_{days}d']
        user_features = user_features.merge(period_events, on='userId', how='left')
        user_features[f'events_last_{days}d'] = user_features[f'events_last_{days}d'].fillna(0)
        
        period_sessions = period_data.groupby('userId')['sessionId'].nunique().reset_index()
        period_sessions.columns = ['userId', f'sessions_last_{days}d']
        user_features = user_features.merge(period_sessions, on='userId', how='left')
        user_features[f'sessions_last_{days}d'] = user_features[f'sessions_last_{days}d'].fillna(0)
    
    # Tendances
    user_features['trend_7d_vs_14d'] = (
        user_features['events_last_7d'] / (user_features['events_last_14d'] - user_features['events_last_7d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    user_features['trend_3d_vs_7d'] = (
        user_features['events_last_3d'] / (user_features['events_last_7d'] - user_features['events_last_3d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    user_features['activity_acceleration'] = user_features['trend_3d_vs_7d'] - user_features['trend_7d_vs_14d']
    
    # Indicateurs binaires
    user_features['is_inactive_7d'] = (user_features['events_last_7d'] == 0).astype(int)
    user_features['is_inactive_14d'] = (user_features['events_last_14d'] == 0).astype(int)
    user_features['is_inactive_21d'] = (user_features['events_last_21d'] == 0).astype(int)
    user_features['has_visited_downgrade'] = (user_features.get('page_downgrade', 0) > 0).astype(int)
    user_features['has_visited_cancel'] = (user_features.get('page_cancel', 0) > 0).astype(int)
    user_features['has_visited_help'] = (user_features.get('page_help', 0) > 0).astype(int)
    user_features['has_errors'] = (user_features.get('page_error', 0) > 0).astype(int)
    user_features['is_low_engagement'] = (user_features['positive_interactions'] < 1).astype(int)
    user_features['is_declining'] = (user_features['trend_7d_vs_14d'] < 0.5).astype(int)
    
    # Features d'interaction
    user_features['level_encoded'] = LabelEncoder().fit_transform(user_features['level'].fillna('unknown'))
    user_features['paid_x_inactive'] = user_features['level_encoded'] * user_features['days_since_last_activity']
    user_features['paid_x_lowengagement'] = user_features['level_encoded'] * user_features['is_low_engagement']
    user_features['tenure_x_inactive'] = user_features['days_since_registration'] * user_features['is_inactive_14d']
    
    return user_features

In [ ]:
def add_profile_distances_for_test(test_features, train_enhanced, n_clusters=4):
    """
    Ajoute les distances aux profils de churners pour les données de test.
    Utilise les centroïdes calculés sur les données d'entraînement.
    """
    clustering_features = [
        'total_events', 'unique_sessions', 'days_since_last_activity',
        'events_per_day', 'positive_interactions', 'thumbs_ratio',
        'page_help', 'error_rate', 'days_since_registration', 'activity_trend'
    ]
    clustering_features = [f for f in clustering_features if f in test_features.columns]
    
    # Entraîner sur les churners du train
    churners_train = train_enhanced[train_enhanced['will_churn_10days'] == 1]
    
    scaler = StandardScaler()
    X_churners = churners_train[clustering_features].fillna(0)
    X_churners_scaled = scaler.fit_transform(X_churners)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_churners_scaled)
    
    # Appliquer sur test
    X_test = test_features[clustering_features].fillna(0)
    X_test_scaled = scaler.transform(X_test)
    
    distances = cdist(X_test_scaled, kmeans.cluster_centers_, metric='euclidean')
    
    for i in range(n_clusters):
        test_features[f'dist_to_churner_profile_{i}'] = distances[:, i]
    
    test_features['min_dist_to_churner'] = distances.min(axis=1)
    test_features['closest_churner_profile'] = distances.argmin(axis=1)
    
    dist_sum = distances.sum(axis=1, keepdims=True)
    proba_profiles = 1 - (distances / (dist_sum + 0.001))
    proba_profiles = proba_profiles / proba_profiles.sum(axis=1, keepdims=True)
    
    for i in range(n_clusters):
        test_features[f'proba_churner_profile_{i}'] = proba_profiles[:, i]
    
    test_features['max_churner_similarity'] = proba_profiles.max(axis=1)
    
    return test_features

In [ ]:
def generate_submission(df_test, train_enhanced, model, feature_cols, reference_date, output_filename='submission.csv'):
    """
    Génère le fichier de soumission Kaggle.
    
    Parameters:
    -----------
    df_test : DataFrame
        Données de test (format événements)
    train_enhanced : DataFrame  
        Données d'entraînement avec features (pour calculer les profils)
    model : sklearn model
        Modèle entraîné
    feature_cols : list
        Liste des features utilisées
    reference_date : datetime
        Date de référence
    output_filename : str
        Nom du fichier de sortie
    
    Returns:
    --------
    DataFrame de soumission
    """
    print("🔄 Création des features pour les données de test...")
    
    # Créer les features
    test_features = create_features_for_submission(df_test, reference_date)
    print(f"   Features de base créées: {test_features.shape}")
    
    # Ajouter les distances aux profils
    test_features = add_profile_distances_for_test(test_features, train_enhanced, n_clusters=4)
    print(f"   Features de distance ajoutées: {test_features.shape}")
    
    # S'assurer que toutes les colonnes nécessaires existent
    for col in feature_cols:
        if col not in test_features.columns:
            test_features[col] = 0
    
    # Préparer X
    X_test = test_features[feature_cols].fillna(0)
    
    # Prédictions
    print("🎯 Génération des prédictions...")
    predictions_proba = model.predict_proba(X_test)[:, 1]
    predictions_binary = (predictions_proba >= 0.5).astype(int)
    
    # Créer le fichier de soumission
    submission = pd.DataFrame({
        'id': test_features['userId'].astype(int),
        'target': predictions_binary
    })
    
    # Sauvegarder
    submission.to_csv(output_filename, index=False)
    
    print(f"\n✅ Fichier de soumission créé: {output_filename}")
    print(f"📊 Statistiques:")
    print(f"   - Total utilisateurs: {len(submission):,}")
    print(f"   - Prédictions churn (1): {submission['target'].sum():,} ({submission['target'].mean()*100:.1f}%)")
    print(f"   - Prédictions non-churn (0): {(submission['target']==0).sum():,}")
    
    return submission, predictions_proba

### 📥 Charger les données de test et générer la soumission

In [ ]:
# ============================================================
# 🎯 GÉNÉRER LA SOUMISSION KAGGLE
# ============================================================

# Charger les données de test (format parquet)
df_test = pd.read_parquet('df_test.parquet')

# Convertir les timestamps
df_test['time'] = pd.to_datetime(df_test['time'])
df_test['registration'] = pd.to_datetime(df_test['registration'])

print(f"📊 Données de test chargées: {len(df_test):,} événements")
print(f"👥 Utilisateurs uniques: {df_test['userId'].nunique():,}")

In [ ]:
# Générer la soumission
submission, probas = generate_submission(
    df_test=df_test,
    train_enhanced=user_df_enhanced,
    model=final_model,
    feature_cols=features_enhanced,
    reference_date=REFERENCE_DATE,
    output_filename='submission.csv'
)

In [ ]:
# Aperçu de la soumission
print("\n📋 Aperçu du fichier de soumission:")
submission.head(20)

In [ ]:
# Distribution des probabilités
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme des probabilités
ax1 = axes[0]
ax1.hist(probas, bins=50, color='#3498db', edgecolor='black', alpha=0.7)
ax1.axvline(x=0.5, color='red', linestyle='--', label='Seuil (0.5)')
ax1.set_xlabel('Probabilité de churn')
ax1.set_ylabel('Nombre d\'utilisateurs')
ax1.set_title('Distribution des Probabilités de Churn', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Distribution des prédictions
ax2 = axes[1]
counts = submission['target'].value_counts()
ax2.bar(['Non-Churn (0)', 'Churn (1)'], counts.values, color=['#2ecc71', '#e74c3c'], edgecolor='black')
ax2.set_ylabel('Nombre d\'utilisateurs')
ax2.set_title('Distribution des Prédictions', fontsize=13, fontweight='bold')

for i, count in enumerate(counts.values):
    ax2.text(i, count + 20, f'{count:,}\n({count/len(submission)*100:.1f}%)', 
             ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('submission_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### 🎚️ Ajuster le seuil de décision (optionnel)

In [ ]:
def generate_submission_with_threshold(probas, user_ids, threshold=0.5, output_filename='submission.csv'):
    """
    Génère une soumission avec un seuil personnalisé.
    """
    predictions = (probas >= threshold).astype(int)
    
    submission = pd.DataFrame({
        'id': user_ids.astype(int),
        'target': predictions
    })
    
    submission.to_csv(output_filename, index=False)
    
    print(f"✅ Soumission avec seuil={threshold} créée: {output_filename}")
    print(f"   Churn prédits: {predictions.sum():,} ({predictions.mean()*100:.1f}%)")
    
    return submission

# Exemple: ajuster le seuil si nécessaire
# submission_adjusted = generate_submission_with_threshold(
#     probas=probas,
#     user_ids=submission['id'],
#     threshold=0.4,  # Seuil plus bas = plus de prédictions de churn
#     output_filename='submission_threshold_04.csv'
# )

---

## 📊 Résumé Final

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    RÉSUMÉ DU PIPELINE                                ║
╚══════════════════════════════════════════════════════════════════════╝
""")

print(f"""
📊 AMÉLIORATION DES PERFORMANCES
{'─'*50}
• Features de base: {len(features_base)}
• Features améliorées: {len(features_enhanced)} (+{len(features_enhanced)-len(features_base)} nouvelles)

🚀 NOUVELLES FEATURES AJOUTÉES:
• Scores composites: frustration_score, disengagement_score, risk_score
• Tendances multi-périodes: 3j, 7j, 14j, 21j
• Distance aux profils de churners: 4 profils
• Indicateurs binaires de risque
• Features d'interaction

📈 GAINS DE PERFORMANCE:
""")

for row in comparison_data:
    print(f"• {row['Modèle']}: {row['AUC Base']:.4f} → {row['AUC Amélioré']:.4f} ({row['Amélioration (%)']:+.2f}%)")

print(f"""
📤 SOUMISSION KAGGLE
{'─'*50}
• Fichier: submission.csv
• Format: id, target (0 ou 1)
• Utilisateurs: {len(submission):,}
• Taux de churn prédit: {submission['target'].mean()*100:.1f}%
""")